<a href="https://colab.research.google.com/github/RobotInventors/BotCodeFinal-RIL/blob/main/PbToTflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install tensorflow
!pip install opencv-python
!pip install numpy

Converting the model from .pt to .tflite

please just put your best.pt file into the runtime ssd

In [7]:
from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO("best.pt")

# Export the model to TFLite format
model.export(format="tflite")  # creates 'yolo11n_float32.tflite'

# Load the exported TFLite model
tflite_model = YOLO("finalModel.tflite")

# Run inference
results = tflite_model("https://ultralytics.com/images/bus.jpg")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 26.6 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/operations/check.py", line 40, in crea

ModuleNotFoundError: No module named 'ultralytics'

Testing the model here using a .png image, please upload the .png into the runtime with the name test.png

In [8]:
import cv2
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="your_model.tflite")
interpreter.allocate_tensors()

# Get model details (input and output tensor information)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to preprocess the image
def preprocess_image(image, target_size=(800, 800)):  # Update this to expected size
    # Resize the image to match the input size of the model
    image_resized = cv2.resize(image, target_size)

    # Normalize the image to the range [0, 1]
    image_normalized = image_resized.astype(np.float32) / 255.0

    # Add batch dimension (1, height, width, channels)
    image_batch = np.expand_dims(image_normalized, axis=0)

    return image_batch

# Function to run inference and draw bounding boxes
def detect_objects(image_path, output_path="output.png"):
    # Read the input image using OpenCV
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return

    # Preprocess the image for the model
    preprocessed_image = preprocess_image(image)

    # Set the tensor input for the model
    interpreter.set_tensor(input_details[0]['index'], preprocessed_image)

    # Run the inference
    interpreter.invoke()

    # Get the output tensor (detections)
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]  # Shape: (6, 13125)
    print(f"Output Data Shape: {output_data.shape}")

    # Check the output shape and reshape accordingly
    # We expect 13125 values for each detection, so we'll reshape to extract meaningful components
    num_detections = output_data.shape[1] // 6  # 6 values per detection (bounding box, class ID, score)
    detections = output_data.reshape((num_detections, 6))  # Reshape to (num_detections, 6)

    threshold = 0.5  # Confidence threshold

    boxes = []
    class_ids = []
    scores = []

    # Extract boxes, class IDs, and scores
    for detection in detections:
        # Extract bounding box, class id, and score from the detection
        ymin, xmin, ymax, xmax = detection[:4]
        class_id = int(detection[4])
        score = detection[5]

        # Only consider detections above the threshold
        if score > threshold:
            boxes.append([ymin, xmin, ymax, xmax])
            class_ids.append(class_id)
            scores.append(score)

    # If there are no detections, print a message
    if len(boxes) == 0:
        print("No detections above the threshold.")
        return

    # Draw the bounding boxes and labels
    for i in range(len(boxes)):
        ymin, xmin, ymax, xmax = boxes[i]
        (h, w, _) = image.shape
        xmin = int(xmin * w)
        xmax = int(xmax * w)
        ymin = int(ymin * h)
        ymax = int(ymax * h)

        # Draw the bounding box on the image
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

        # Display the class label (optional)
        label = f"Class: {class_ids[i]} Score: {scores[i]:.2f}"
        cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the resulting image with bounding boxes
    cv2.imwrite(output_path, image)

    # Optionally, display the image
    cv2.imshow("Detected Objects", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage: Run object detection on 'test.png' and save the result as 'output.png'
image_path = 'test.jpg'  # Replace with the path to your image
detect_objects(image_path, output_path="output.png")




Output tensor 0: {'name': 'Identity', 'index': 407, 'shape': array([    1,     6, 13125], dtype=int32), 'shape_signature': array([    1,     6, 13125], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
Output Data Shape: (6, 13125)
No detections above the threshold.


Converting the Float32 Model to Int8 for use with google coral